In [35]:
using Pkg
using Knet: Knet, param, param0, @diff, grad,RNN, params, KnetArray, conv4, Data, relu, dropout
using Random: shuffle!
using IterTools: ncycle
using TestImages,Images, ImageView
using PyCall
@pyimport pickle
@pyimport numpy

In [2]:
word_to_ix = []
label_to_ix = []

instructions = []
labels = []
sentence = []
words = []

wdict = Dict()
tdict = Dict()
w2i(x) = get!(wdict, x, 1+length(wdict))
t2i(x) = get!(tdict, x, 1+length(tdict))
UNK = w2i("<unk>")

1

In [3]:
open("instructions.txt") do f
    for line in eachline(f)
#         line = split(strip(line),",")
#         label = [line[2]]
#         sentence = map(x-> lowercase(x),split(strip(line[1])," "))
#         append!(instructions,(sentence,label))
        sentence, label = split(strip(lowercase(line)), ",")
        wordids = w2i.(split(sentence))
        labelid = t2i(label)
        push!(instructions, (wordids, labelid))
        push!(words, split(strip(sentence), " "))
    end
#      print(instructions)
    print(words)
end

Any[SubString{String}["climb", "down", "the", "ladder"], SubString{String}["climb", "down", "the", "ladder"], SubString{String}["climb", "down", "the", "ladder"], SubString{String}["climb", "down", "the", "ladder"], SubString{String}["climb", "down", "the", "ladder"], SubString{String}["climb", "up", "the", "ladder"], SubString{String}["climb", "up", "the", "ladder"], SubString{String}["climb", "up", "the", "ladder"], SubString{String}["climb", "up", "the", "ladder"], SubString{String}["get", "the", "key"], SubString{String}["get", "the", "key"], SubString{String}["get", "the", "sword"], SubString{String}["get", "the", "torch"], SubString{String}["go", "between", "the", "lasers"], SubString{String}["go", "between", "the", "lasers"], SubString{String}["go", "between", "the", "lasers"], SubString{String}["go", "between", "the", "lasers"], SubString{String}["go", "to", "the", "bottom", "of", "the", "room"], SubString{String}["go", "to", "the", "bottom", "of", "the", "room"], SubString{Str

In [4]:
print(instructions[5])

([2, 3, 4, 5], 1)

In [5]:
function prepare_sentence(sent, to_ix)
    sent = split(strip(lowercase(sent))," ")
    idxs = [to_ix[w] for w in sent]
    return torch.tensor(idxs)
end

prepare_sentence (generic function with 1 method)

In [6]:
EMBEDDING_DIM = 20
HIDDEN_DIM_LSTM = 10
VOCAB_SIZE = length(word_to_ix)
LABEL_SIZE = length(label_to_ix)

# text_model = LSTMClassifier()
# image_model = ConvNetClassifier()
# loss_function = nn.CosineEmbeddingLoss()
# optimizer1 = optim.SGD(text_model.parameters(), lr = 0.001)
# optimizer2 = optim.SGD(image_model.parameters(), lr = 0.001)

0

In [7]:
using PyCall
@pyimport pickle

function mypickle(filename, obj)
    out = open(filename,"w")
    pickle.dump(obj, out)
    close(out)
 end

function myunpickle(filename)
    r = nothing
    @pywith pybuiltin("open")(filename,"rb") as f begin
        r = pickle.load(f)
    end
    return r
end

myunpickle (generic function with 1 method)

In [8]:
l = myunpickle("beating-atari-with-natural-language-guided-rl/dataset/dataset_true.pickle")

347-element Array{Tuple{Tuple{Array{UInt8,3},Array{UInt8,3}},String},1}:
 (([0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00]

[0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00]

[0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00], [0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00]

[0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00]

[0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00]), "Climb down the ladder")       
 (([0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00]

[0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00]

[0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00; … ; 0x00 0x00 … 0x00 0x00; 0x00 0x00 … 0x00 0x00], [0x00 

In [9]:
wdict2 = Dict()
tdict2 = Dict()
w2i2(x) = get!(wdict2, x, 1+length(wdict2))
t2i2(x) = get!(tdict2, x, 1+length(tdict2))
UNK2 = w2i2("<unk>")

1

In [10]:
# function GetPreProcessedImageArray(img, cropfromtop::Float64, cropfrombottom::Float64, target_height::Int, target_width::Int)
#     img = img ./ 255
#     height, width, channels = size(img)
#     state_image = permutedims(img,(1,2,3))
#     state_image2 = (state_image[:,Int(height*cropfromtop)+1:Int((1-cropfrombottom)*height),:])
#     grayscale = Gray.(state_image2)
# #     reshape(Float64.(imresize(grayscale, (1,target_height, target_width))), (target_width,target_height))
# end

In [11]:
images = []
im_sentences = []
for i in 1:length(l)
    (f1,f2), sent = l[i]
    #both = numpy.hstack((f1,f2))
    # append!(images,split(strip(sent)," "))
    #resf1 = permutedims((f1[:,:,:])/255,(3,1,2))
    #imshow(resf1)
    #resf2 = permutedims((f1[:,:,:])/255,(3,1,2))
    both = hcat(f1,f2)
#     tst = GetPreProcessedImageArray(both, 0.0, 0.0, 210,320)
#     println(size(tst))
#     imshow(tst)
#     break
    
#     grayf1 = Gray.(resf1)
#     grayf2 = Gray.(resf2)
#     donef1 = reshape(Float64.(imresize(grayf1, (1,210, 60))), 210*60)
#     donef2 = reshape(Float64.(imresize(grayf2, (1,210, 60))), 210*60)
#     both = hcat(donef1,donef2)
#     both = hcat(grayf1,grayf2)
#     print(size(grayf1))
  

#     When we change it to (210,320) it will work better I guess, now its just changed to Float64
    b3 = reshape(Float32.(both), (210,320,3))
#         print(typeof(b3))
    
    wordids = w2i2.(split(sent))
    #labelid = t2i(label)
    push!(im_sentences, wordids)
    push!(images,b3)
end
# println(images)

In [12]:
typeof(images[1])

Array{Float32,3}

In [13]:
# Define a convolutional layer:
struct Conv; w; b; f; p; end
(c::Conv)(x) = c.f.(pool(conv4(c.w, dropout(x,c.p)) .+ c.b))
Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Conv

In [14]:
struct Convp; w; b; f; p; end
(c::Convp)(x) = c.f.(conv4(c.w, dropout(x,c.p)) .+ c.b)
Convp(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Convp(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Convp

In [15]:
# Redefine dense layer (See mlp.ipynb):
struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [16]:
struct Densep; w; b; f; p; end
(d::Densep)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Densep(i::Int,o::Int,f=relu;pdrop=0) = Densep(param(o,i), param0(o), f, pdrop)

Densep

In [17]:
# Let's define a chain of layers
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)
(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d)

In [18]:
struct Embed; w; end
Embed(vocabsize::Int,embedsize::Int) = Embed(param(embedsize,vocabsize))
(e::Embed)(x) = e.w[:,x]

In [19]:
Tagger(vocab,embed,hidden,output)=  # biRNN Tagger
    Chain(Embed(vocab,embed),RNN(embed,hidden,rnnType=:relu,bidirectional=true),Dense(2hidden,output));

In [20]:
#Lets get a try in

testChain = Chain(Conv(5,5,3,32),#103*158
    Conv(5,5,32,32),#49*77
    Conv(4,4,32,64),#23*37
    Convp(3,3,64,64),#21*35*64 = 47040
    Dense(47040,10,pdrop=0.3))
summary.(l.w for l in testChain.layers)


5-element Array{String,1}:
 "5×5×3×32 AutoGrad.Param{Array{Float32,4}}" 
 "5×5×32×32 AutoGrad.Param{Array{Float32,4}}"
 "4×4×32×64 AutoGrad.Param{Array{Float32,4}}"
 "3×3×64×64 AutoGrad.Param{Array{Float32,4}}"
 "10×47040 AutoGrad.Param{Array{Float32,2}}" 

In [21]:
# testChain(images[1])

In [22]:
embedlayer = Embed(length(instructions),128)


Embed(P(Array{Float32,2}(128,60)))

In [23]:
# For running experiments we will use the Adam algorithm which typically converges faster than SGD.
function trainresults(file,maker,savemodel)
    if (print("Train from scratch? "); readline()[1]=='y')
        model = maker()
        results = ((nll(model,dtst), zeroone(model,dtst))
                   for x in takenth(progress(adam(model,ncycle(dtrn,5))),100))
        results = reshape(collect(Float32,flatten(results)),(2,:))
        Knet.save(file,"model",(savemodel ? model : nothing),"results",results)
        Knet.gc() # To save gpu memory
    else
        isfile(file) || download("http://people.csail.mit.edu/deniz/models/tutorial/$file",file)
        model,results = Knet.load(file,"model","results")
    end
    println(minimum(results,dims=2))
    return model,results
end

trainresults (generic function with 1 method)

In [30]:
VOCABSIZE = length(instructions)
EMBEDSIZE = 128
HIDDENSIZE = 128
OUTPUTSIZE = length(instructions);

In [36]:
t2maker() = Tagger(VOCABSIZE,EMBEDSIZE,HIDDENSIZE,OUTPUTSIZE)
(t2,r2) = trainresults("tagger113c.jld2",t2maker,true);

Train from scratch? stdin> y


UndefVarError: UndefVarError: dtrn not defined

In [26]:
# struct LSTMClas
#     embeddings
#     lstm
#     fullyconnected
#     hidden
# end

# function LSTMClas()
#     embeddings = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM) 
#     lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM_LSTM)
#     fullyconnected = nn.Linear(HIDDEN_DIM_LSTM, 10)
# end
# #NOt done


In [27]:
# class LSTMClassifier(nn.Module):

# 	def __init__(self):
		
# 		super(LSTMClassifier, self).__init__()

# 		self.embeddings = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
# 		self.lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM_LSTM)
# 		self.fullyconnected = nn.Linear(HIDDEN_DIM_LSTM, 10)
# 		self.hidden = self.init_hidden()

# 	def init_hidden(self):
# 		# the first is the hidden h
# 		# the second is the cell  c
# 		return (autograd.Variable(torch.zeros(1, 1, HIDDEN_DIM_LSTM)),
#                 autograd.Variable(torch.zeros(1, 1, HIDDEN_DIM_LSTM)))

# 	def forward(self, sentence):

# 		embeds = self.embeddings(sentence)
# 		x = embeds.view(len(sentence), 1, -1)
# 		lstm_out, self.hidden = self.lstm(x, self.hidden)
# 		#print (lstm_out)
# 		y  = self.fullyconnected(lstm_out[-1])
# 		# log_probs = F.log_softmax(y)
# 		#print (y)
# 		return y
